<a href="https://colab.research.google.com/github/BooEiEi/project/blob/main/multiple_imputation_20per.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #นำเข้าข้อมูลจาก drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project'# ชี้ dirve ที่เราต้องการ
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

Mounted at /content/drive


In [ ]:
data = pd.read_csv(os.path.join(path, 'data_telomere.csv'))
data = data.drop(['Unnamed: 0'], axis=1)
data

,Sex,AgeSYear,telomere length (kb),Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,58.0,7.30,47.2,156.0,19.4,112.0,64.0,70.0,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
1,2,57.0,6.36,47.4,153.0,20.2,114.0,75.0,69.0,19.3,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
2,2,24.0,9.71,53.2,157.0,21.6,103.0,69.0,78.0,21.3,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
3,2,62.0,6.17,54.4,165.0,20.0,95.0,75.0,83.0,21.6,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
4,2,86.0,4.55,49.0,154.0,20.7,129.0,79.0,83.0,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2,25.0,8.98,47.0,165.0,17.3,104.0,63.0,89.0,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
418,2,31.0,8.23,72.2,168.0,25.6,123.0,89.0,69.0,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
419,2,32.0,8.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
420,1,29.0,8.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [ ]:
data = data.drop(['telomere length (kb) '], axis=1)
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,58.0,47.2,156.0,19.4,112.0,64.0,70.0,NaN,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
1,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
2,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
3,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
4,2,86.0,49.0,154.0,20.7,129.0,79.0,83.0,NaN,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2,25.0,47.0,165.0,17.3,104.0,63.0,89.0,NaN,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
418,2,31.0,72.2,168.0,25.6,123.0,89.0,69.0,NaN,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
419,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
420,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [ ]:
P = 0
p1 = 0
list_mis = []
for i in data:
  if data[i].isna().sum() == 0 :
    continue
  else:
    ms = data[i].isna().sum()
    P = (ms/1734).round(14)
    p1 = p1 + P
    print(f'ความน่าจะเป็นของ {i} = {P}')
    list_mis.append(P)
print('****************************************')
print(f'ความน่าจะเป็นทั้งหมด = {p1}')

ความน่าจะเป็นของ Weight = 0.00461361014994
ความน่าจะเป็นของ Height = 0.00461361014994
ความน่าจะเป็นของ BMI = 0.00461361014994
ความน่าจะเป็นของ Systolic = 0.00461361014994
ความน่าจะเป็นของ Diastolic = 0.00519031141869
ความน่าจะเป็นของ HeartRate = 0.02941176470588
ความน่าจะเป็นของ SMM = 0.12975778546713
ความน่าจะเป็นของ Fat Mass = 0.12975778546713
ความน่าจะเป็นของ % Body fat = 0.12975778546713
ความน่าจะเป็นของ waist to hip radio = 0.12975778546713
ความน่าจะเป็นของ abdominal circumference = 0.13033448673587
ความน่าจะเป็นของ visceral fat area = 0.12975778546713
ความน่าจะเป็นของ 001_Glucose = 0.00057670126874
ความน่าจะเป็นของ 004_Uric Acid = 0.05363321799308
ความน่าจะเป็นของ 012_ALT = 0.00057670126874
ความน่าจะเป็นของ 013_Alkaline Phos = 0.01672433679354
ความน่าจะเป็นของ 071_HbA1c = 0.09630911188005
****************************************
ความน่าจะเป็นทั้งหมด = 1.0


In [ ]:
data.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [ ]:
data1 = pd.read_csv(os.path.join(path, 'DFforCreateNaN.csv'))
data1 = data1.drop(['Unnamed: 0'], axis=1)
data1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [ ]:
data1.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [ ]:
data1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [ ]:
list1 = []
for i in data:
  if data[i].isna().sum() == 0:
    continue
  else: 
    list1.append(i)
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [ ]:
list_mis

[0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00519031141869,
 0.02941176470588,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.13033448673587,
 0.12975778546713,
 0.00057670126874,
 0.05363321799308,
 0.00057670126874,
 0.01672433679354,
 0.09630911188005]

# 20 %

## แทนค่า NaN

In [ ]:
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'random_column_{i}'] = np.random.choice(list1,int((153*17)*(20/100)),p=list_mis)
  globals()[f'random_column_{i}'] = list(globals()[f'random_column_{i}'])


In [ ]:
for i in range(1,1001):
  for j in globals()[f'random_column_{i}']:
    globals()[f'count_{i}'] = []
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Weight'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Height'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('BMI'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Systolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Diastolic'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('HeartRate'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('SMM'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('FatMass'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Bodyfat'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('waisttohip'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('abdominal'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('visceral'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('001_Glucose'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('UricAcid'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('012_ALT'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('Alkaline_Phos'))
    globals()[f'count_{i}'].append(globals()[f'random_column_{i}'].count('071_HbA1c'))

In [ ]:
list2 = []
list3 = []

for i in range(153):
  m = 1/153
  list2.append(i)
  list3.append(m)

In [ ]:
# random row ไม่ซ้ำกัน
for i in range(1,1001):
  np.random.seed(i)
  globals()[f'dt20_{i}'] = data1.copy()
  for j,k in zip(list1,globals()[f'count_{i}']):
    globals()[f'dt20_{i}'][j].ravel()[np.random.choice(list2,k,p= list3, replace=False)] = np.nan

In [ ]:
dt20_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,NaN,153.0,20.2,114.0,75.0,69.0,NaN,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,NaN
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,NaN,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,NaN
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,NaN,NaN,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,NaN,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,NaN,NaN,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,NaN
4,1,46.0,72.3,170.0,25.0,133.0,NaN,107.0,29.2,NaN,...,0.9,NaN,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,NaN,30.5,NaN,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,NaN,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,NaN,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,NaN
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,NaN,NaN,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation

### Imputation using Mean

In [40]:
from sklearn.impute import SimpleImputer

In [41]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean_{i}'] = mean_imputer.fit_transform(globals()[f'dt20_{i}'].copy())
  globals()[f'MSE_MEAN20_{i}'] = mean_squared_error(data1,globals()[f'imputed_mean_{i}'])
t_end = time.time()
t_MEAN20 = (t_end- t_start)

In [43]:
t_MEAN20

3.8026163578033447

In [42]:
mse_MEAN20_total = 0 
for i in range(1,1001):
  mse_MEAN20_total = mse_MEAN20_total + globals()[f'MSE_MEAN20_{i}']
AMSE_mean20 = mse_MEAN20_total/1000
print(f'AMSE_mean = {AMSE_mean20}')

AMSE_mean = 23.096538103054137


### Imputation using the Miss Forest

In [ ]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 6.5 MB/s eta 0:00:00


In [ ]:
t_start = time.time()
MF = MissForest()
for i in range(1,1001):
  globals()[f'imputed_MF_{i}'] = MF.fit_transform(globals()[f'dt20_{i}'].copy())
  globals()[f'MSE_MF20_{i}'] = mean_squared_error(data1,globals()[f'imputed_MF_{i}'])
t_end = time.time()
t_MF20 = (t_end- t_start)

In [18]:
t_MF20

15988.587602853775

In [19]:
mse_MF20_total = 0 
for i in range(1,1001):
  mse_MF20_total = mse_MF20_total + globals()[f'MSE_MF20_{i}']
AMSE_MF20 = mse_MF20_total/1000
print(f'AMSE_MF = {AMSE_MF20}')

AMSE_MF = 6.078007382722867


### Imputation using the KNN

In [20]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [21]:
t_start = time.time()
KNN = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN_{i}'] = KNN.fit_transform(globals()[f'dt20_{i}'].copy())
  globals()[f'MSE_KNN20_{i}'] = mean_squared_error(data1,globals()[f'imputed_KNN_{i}'])
t_end = time.time()
t_KNN20 = (t_end- t_start)

In [22]:
t_KNN20

15.902750730514526

In [23]:
mse_KNN20_total = 0 
for i in range(1,1001):
  mse_KNN20_total = mse_KNN20_total + globals()[f'MSE_KNN20_{i}']
AMSE_KNN20 = mse_KNN20_total/1000
print(f'AMSE_KNN = {AMSE_KNN20}')

AMSE_KNN = 17.77258916459049


### Imputation using the MICE

In [24]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_LR_{i}'] = imp.fit_transform(globals()[f'dt20_{i}'].copy())
  globals()[f'MSE_LR20_{i}'] = mean_squared_error(data1,globals()[f'imputed_LR_{i}'])
t_end = time.time()
t_LR20 = (t_end- t_start)

In [35]:
t_LR20

158.97983360290527

In [39]:
mse_LR20_total = 0 
for i in range(1,1001):
  mse_LR20_total = mse_LR20_total + globals()[f'MSE_LR20_{i}']
AMSE_LR20 = mse_LR20_total/1000
print(f'AMSE_LR = {AMSE_LR20}')

AMSE_LR = 8.517123877011059


#### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_SVM_{i}'] = svm_svr.fit_transform(globals()[f'dt20_{i}'].copy())
  globals()[f'MSE_SVM20_{i}'] = mean_squared_error(data1,globals()[f'imputed_SVM_{i}'])
t_end = time.time()
t_SVM20 = (t_end- t_start)

In [38]:
t_SVM20

166.69177722930908

In [37]:
mse_SVM20_total = 0 
for i in range(1,1001):
  mse_SVM20_total = mse_SVM20_total + globals()[f'MSE_SVM20_{i}']
AMSE_SVM20 = mse_SVM20_total/1000
print(f'AMSE_SVM = {AMSE_SVM20}')

AMSE_SVM = 24.76551355423316


### Imputation using the EM

In [ ]:
!pip install impyute
import impyute as impy

In [32]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_EM_{i}'] = impy.em(globals()[f'dt20_{i}'].values.copy())
  globals()[f'MSE_EM20_{i}'] = mean_squared_error(data1,globals()[f'imputed_EM_{i}'])
t_end = time.time()
t_EM20 = (t_end- t_start)

In [36]:
t_EM20

160.01999068260193

In [34]:
mse_EM20_total = 0 
for i in range(1,1001):
  mse_EM20_total = mse_EM20_total + globals()[f'MSE_EM20_{i}']
AMSE_EM20 = mse_EM20_total/1000
print(f'AMSE_EM = {AMSE_EM20}')

AMSE_EM = 45.74857607341226
